# 🕑 DDL: Tabela Silver - `tb_chamado_log`

**Origem:** Tabela Bronze `chamados_hora`
**Domínio:** Auditoria e SLAs de Tempo

## ⚙️ Decisões de Governança e Modelagem

### 1. Padronização Temporal (`dh_`)
Alteramos os prefixos de colunas de data/hora para indicar precisão:
- **`data_` → `dh_` (Data Hora):** Indica que o campo é um `TIMESTAMP` completo, não apenas uma data (`dt_`). Isso é crucial para cálculos de SLA.

### 2. Mapeamento de Schema (Bronze → Silver)
| Coluna Bronze | Coluna Silver | Tipo Novo | Justificativa |
|---|---|---|---|
| `id_chamado` | `cd_chamado` | `BIGINT` | PK e vínculo principal. |
| `id_cliente` | `cd_cliente` | `STRING` | Identificação do cliente. |
| `data_abertura`| `dh_abertura`| `TIMESTAMP` | Abertura do ticket. |
| `data_inicio` | `dh_inicio` | `TIMESTAMP` | Início do atendimento humano/bot. |
| `data_fim` | `dh_fim` | `TIMESTAMP` | Encerramento do ticket. |

### 3. Constraints
- **Primary Key:** `pk_chamado_log` (`cd_chamado`). Nota: Assume relanção 1:1 com a tabela fato de chamados para logs sumarizados.

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import TimestampType

CATALOGO_DESTINO = "v_credit"
SCHEMA_DESTINO = "silver"
TABELA_DESTINO = "tb_chamado_log"

nome_tabela_destino = f"{CATALOGO_DESTINO}.{SCHEMA_DESTINO}.{TABELA_DESTINO}"

In [0]:
query = f"""
CREATE OR REPLACE TABLE {nome_tabela_destino} (
    cd_chamado BIGINT COMMENT 'Referência ao chamado principal.',
    cd_cliente STRING COMMENT 'Referência ao cliente.',
    dh_abertura TIMESTAMP COMMENT 'Timestamp de abertura.',
    dh_inicio TIMESTAMP COMMENT 'Timestamp de início.',
    dh_fim TIMESTAMP COMMENT 'Timestamp de fim.',
    dt_ingestion TIMESTAMP,
    dc_origem STRING,
    CONSTRAINT pk_chamado_log PRIMARY KEY (cd_chamado)
)
COMMENT 'Tabela silver de logs de horas (abertura, início e fim) de chamados';
"""

spark.sql(query)
print(f"✅ Tabela {nome_tabela_destino} criada com sucesso!")